In [ ]:
import logging
import warnings

import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymc as pm
import seaborn as sns

import pcntoolkit.util.output
from pcntoolkit import (
    HBR,
    BsplineBasisFunction,
    NormalLikelihood,
    NormativeModel,
    NormData,
    load_fcon1000,
    make_prior,
)
from pcntoolkit.util.model_comparison import compare_hbr_models

sns.set_style("darkgrid")

# Suppress some annoying warnings and logs
pymc_logger = logging.getLogger("pymc")

pymc_logger.setLevel(logging.WARNING)
pymc_logger.propagate = False

warnings.simplefilter(action="ignore", category=FutureWarning)
pd.options.mode.chained_assignment = None  # default='warn'
pcntoolkit.util.output.Output.set_show_messages(True)

In [2]:
# Download an example dataset
norm_data: NormData = load_fcon1000()

# Select only a few features
features_to_model = [
    "WM-hypointensities",
    "Right-Lateral-Ventricle",
    # "Right-Amygdala",
    # "CortexVol",
]
norm_data = norm_data.sel({"response_vars": features_to_model})

# Split into train and test sets
train, test = norm_data.train_test_split()


Process: 94853 - 2025-08-18 10:35:55 - Dataset "fcon1000" created.
    - 1078 observations
    - 1078 unique subjects
    - 1 covariates
    - 217 response variables
    - 2 batch effects:
    	sex (2)
	site (23)
    


In [3]:
mu1 = make_prior(
    # Mu is linear because we want to allow the mean to vary as a function of the covariates.
    linear=True,
    # The slope coefficients are assumed to be normally distributed, with a mean of 0 and a standard deviation of 10.
    slope=make_prior(dist_name="Normal", dist_params=(0.0, 10.0)),
    # The intercept is not random, because we want to compare to a model with random intercept
    intercept=make_prior(
        dist_name = "Normal", dist_params = (0.0, 2.0),
    ),
    # We use a B-spline basis function to allow for non-linearity in the mean.
    basis_function=BsplineBasisFunction(basis_column=0, nknots=5, degree=3),
)
sigma1 = make_prior(
    # Sigma is also linear, because we want to allow the standard deviation to vary as a function of the covariates: heteroskedasticity.
    linear=True,
    # The slope coefficients are assumed to be normally distributed, with a mean of 0 and a standard deviation of 2.
    slope=make_prior(dist_name="Normal", dist_params=(0.0, 2.0)),
    # The intercept is not random, because we assume the intercept of the variance to be the same for all sites and sexes.
    intercept=make_prior(dist_name="Normal", dist_params=(1.0, 1.0)),
    # We use a B-spline basis function to allow for non-linearity in the standard deviation.
    basis_function=BsplineBasisFunction(basis_column=0, nknots=5, degree=3),
    # We use a softplus mapping to ensure that sigma is strictly positive.
    mapping="softplus",
    # We scale the softplus mapping by a factor of 3, to avoid spikes in the resulting density.
    # The parameters (a, b, c) provided to a mapping f are used as: f_abc(x) = f((x - a) / b) * b + c
    # This basically provides an affine transformation of the softplus function.
    # a -> horizontal shift
    # b -> scaling
    # c -> vertical shift
    # You can leave c out, and it will default to 0.
    mapping_params=(0.0, 3.0),
)
# Set the likelihood with the priors we just created.
likelihood1 = NormalLikelihood(mu1, sigma1)

template_hbr_1 = HBR(
    name="template",
    # The number of cores to use for sampling.
    cores=16,
    # Whether to show a progress bar during the model fitting.
    progressbar=True,
    # The number of draws to sample from the posterior per chain.
    draws=1500,
    # The number of tuning steps to run.
    tune=500,
    # The number of MCMC chains to run.
    chains=4,
    # The sampler to use for the model.
    nuts_sampler="nutpie",
    # The likelihood function to use for the model.
    likelihood=likelihood1,
)
model1 = NormativeModel(
    # The regression model to use for the normative model.
    template_regression_model=template_hbr_1,
    # Whether to save the model after fitting.
    savemodel=True,
    # Whether to evaluate the model after fitting.
    evaluate_model=True,
    # Whether to save the results after evaluation.
    saveresults=True,
    # Whether to save the plots after fitting.
    saveplots=False,
    # The directory to save the model, results, and plots.
    save_dir="resources/compare_hbr/model1",
    # The scaler to use for the input data. Can be either one of "standardize", "minmax", "robminmax", "none"
    inscaler="standardize",
    # The scaler to use for the output data. Can be either one of "standardize", "minmax", "robminmax", "none"
    outscaler="standardize",
)

In [4]:
mu2 = make_prior(
    # Mu is linear because we want to allow the mean to vary as a function of the covariates.
    linear=True,
    # The slope coefficients are assumed to be normally distributed, with a mean of 0 and a standard deviation of 10.
    slope=make_prior(dist_name="Normal", dist_params=(0.0, 10.0)),
    # The intercept is random, because we expect the intercept to vary between sites and sexes.
    intercept=make_prior(
        random=True,
        # Mu is the mean of the intercept, which is normally distributed with a mean of 0 and a standard deviation of 1.
        mu=make_prior(dist_name="Normal", dist_params=(0.0, 1.0)),
        # Sigma is the scale at which the intercepts vary. It is a positive parameter, so we have to map it to the positive domain.
        sigma=make_prior(dist_name="Normal", dist_params=(0.0, 1.0), mapping="softplus", mapping_params=(0.0, 3.0)),
    ),
    # We use a B-spline basis function to allow for non-linearity in the mean.
    basis_function=BsplineBasisFunction(basis_column=0, nknots=5, degree=3),
)
sigma2 = make_prior(
    # Sigma is also linear, because we want to allow the standard deviation to vary as a function of the covariates: heteroskedasticity.
    linear=True,
    # The slope coefficients are assumed to be normally distributed, with a mean of 0 and a standard deviation of 2.
    slope=make_prior(dist_name="Normal", dist_params=(0.0, 2.0)),
    # The intercept is not random, because we assume the intercept of the variance to be the same for all sites and sexes.
    intercept=make_prior(dist_name="Normal", dist_params=(1.0, 1.0)),
    # We use a B-spline basis function to allow for non-linearity in the standard deviation.
    basis_function=BsplineBasisFunction(basis_column=0, nknots=5, degree=3),
    # We use a softplus mapping to ensure that sigma is strictly positive.
    mapping="softplus",
    # We scale the softplus mapping by a factor of 3, to avoid spikes in the resulting density.
    # The parameters (a, b, c) provided to a mapping f are used as: f_abc(x) = f((x - a) / b) * b + c
    # This basically provides an affine transformation of the softplus function.
    # a -> horizontal shift
    # b -> scaling
    # c -> vertical shift
    # You can leave c out, and it will default to 0.
    mapping_params=(0.0, 3.0),
)
# Set the likelihood with the priors we just created.
likelihood2 = NormalLikelihood(mu2, sigma2)

template_hbr_2 = HBR(
    name="template",
    # The number of cores to use for sampling.
    cores=16,
    # Whether to show a progress bar during the model fitting.
    progressbar=True,
    # The number of draws to sample from the posterior per chain.
    draws=1500,
    # The number of tuning steps to run.
    tune=500,
    # The number of MCMC chains to run.
    chains=4,
    # The sampler to use for the model.
    nuts_sampler="nutpie",
    # The likelihood function to use for the model.
    likelihood=likelihood2,
)
model2 = NormativeModel(
    # The regression model to use for the normative model.
    template_regression_model=template_hbr_2,
    # Whether to save the model after fitting.
    savemodel=True,
    # Whether to evaluate the model after fitting.
    evaluate_model=True,
    # Whether to save the results after evaluation.
    saveresults=True,
    # Whether to save the plots after fitting.
    saveplots=False,
    # The directory to save the model, results, and plots.
    save_dir="resources/compare_hbr/model2",
    # The scaler to use for the input data. Can be either one of "standardize", "minmax", "robminmax", "none"
    inscaler="standardize",
    # The scaler to use for the output data. Can be either one of "standardize", "minmax", "robminmax", "none"
    outscaler="standardize",
)

In [5]:
model1.fit_predict(train, test)
model2.fit_predict(train, test)

Process: 94853 - 2025-08-18 10:35:55 - Fitting models on 2 response variables.
Process: 94853 - 2025-08-18 10:35:55 - Fitting model for WM-hypointensities.


Progress,Draws,Divergences,Step Size,Gradients/Draw
,2000,0,0.09,63
,2000,0,0.09,127
,2000,0,0.09,511
,2000,0,0.09,383


Process: 94853 - 2025-08-18 10:36:16 - Fitting model for Right-Lateral-Ventricle.


Progress,Draws,Divergences,Step Size,Gradients/Draw
,2000,0,0.11,63
,2000,0,0.11,31
,2000,0,0.12,255
,2000,0,0.11,95


Process: 94853 - 2025-08-18 10:36:32 - Making predictions on 2 response variables.
Process: 94853 - 2025-08-18 10:36:32 - Computing z-scores for 2 response variables.
Process: 94853 - 2025-08-18 10:36:32 - Computing z-scores for Right-Lateral-Ventricle.
Process: 94853 - 2025-08-18 10:36:32 - Computing z-scores for WM-hypointensities.
Process: 94853 - 2025-08-18 10:36:33 - Computing centiles for 2 response variables.
Process: 94853 - 2025-08-18 10:36:33 - Computing centiles for Right-Lateral-Ventricle.
Process: 94853 - 2025-08-18 10:36:34 - Computing centiles for WM-hypointensities.
Process: 94853 - 2025-08-18 10:36:35 - Computing log-probabilities for 2 response variables.
Process: 94853 - 2025-08-18 10:36:35 - Computing log-probabilities for Right-Lateral-Ventricle.
Process: 94853 - 2025-08-18 10:36:36 - Computing log-probabilities for WM-hypointensities.
Process: 94853 - 2025-08-18 10:36:36 - Computing yhat for 2 response variables.
Process: 94853 - 2025-08-18 10:36:36 - Saving model

Progress,Draws,Divergences,Step Size,Gradients/Draw
,2000,11,0.10,31
,2000,10,0.11,31
,2000,14,0.10,159
,2000,15,0.10,31


Process: 94853 - 2025-08-18 10:36:58 - Fitting model for Right-Lateral-Ventricle.


Progress,Draws,Divergences,Step Size,Gradients/Draw
,2000,34,0.13,31
,2000,2,0.13,95
,2000,19,0.13,63
,2000,1,0.12,63


Process: 94853 - 2025-08-18 10:37:10 - Making predictions on 2 response variables.
Process: 94853 - 2025-08-18 10:37:10 - Computing z-scores for 2 response variables.
Process: 94853 - 2025-08-18 10:37:10 - Computing z-scores for Right-Lateral-Ventricle.
Process: 94853 - 2025-08-18 10:37:10 - Computing z-scores for WM-hypointensities.
Process: 94853 - 2025-08-18 10:37:11 - Computing centiles for 2 response variables.
Process: 94853 - 2025-08-18 10:37:11 - Computing centiles for Right-Lateral-Ventricle.
Process: 94853 - 2025-08-18 10:37:12 - Computing centiles for WM-hypointensities.
Process: 94853 - 2025-08-18 10:37:14 - Computing log-probabilities for 2 response variables.
Process: 94853 - 2025-08-18 10:37:14 - Computing log-probabilities for Right-Lateral-Ventricle.
Process: 94853 - 2025-08-18 10:37:14 - Computing log-probabilities for WM-hypointensities.
Process: 94853 - 2025-08-18 10:37:15 - Computing yhat for 2 response variables.
Process: 94853 - 2025-08-18 10:37:16 - Saving model

<xarray.NormData> Size: 66kB
Dimensions:            (observations: 216, response_vars: 2, covariates: 1,
                        batch_effect_dims: 2, centile: 5, statistic: 11)
Coordinates:
  * observations       (observations) int64 2kB 756 769 692 616 ... 751 470 1043
  * response_vars      (response_vars) <U23 184B 'WM-hypointensities' 'Right-...
  * covariates         (covariates) <U3 12B 'age'
  * batch_effect_dims  (batch_effect_dims) <U4 32B 'sex' 'site'
  * centile            (centile) float64 40B 0.05 0.25 0.5 0.75 0.95
  * statistic          (statistic) <U8 352B 'EXPV' 'MACE' ... 'SMSE' 'ShapiroW'
Data variables:
    subjects           (observations) object 2kB 'Munchen_sub96752' ... 'Quee...
    Y                  (observations, response_vars) float64 3kB 2.721e+03 .....
    X                  (observations, covariates) float64 2kB 63.0 ... 23.0
    batch_effects      (observations, batch_effect_dims) <U17 29kB 'F' ... 'Q...
    Z                  (observations, response_vars) float64 3kB 0.6046 ... 1...
    centiles           (centile, observations, response_vars) float64 17kB -5...
    logp               (observations, response_vars) float64 3kB -1.702 ... -...
    Yhat               (observations, response_vars) float64 3kB 1.832e+03 .....
    statistics         (response_vars, statistic) float64 176B 0.3631 ... 0.8767
Attributes:
    real_ids:                       True
    is_scaled:                      False
    name:                           fcon1000_test
    unique_batch_effects:           {np.str_('sex'): [np.str_('F'), np.str_('...
    batch_effect_counts:            defaultdict(<function NormData.register_b...
    covariate_ranges:               {np.str_('age'): {'min': np.float64(7.88)...
    batch_effect_covariate_ranges:  {np.str_('sex'): {np.str_('F'): {np.str_(...

In [6]:
# Delete references to model objects to ensure what follows will work for models saved to disk too
del model1
del model2

In [7]:
dct = {"model1": "resources/compare_hbr/model1", "model2": "resources/compare_hbr/model2"}
comparison = compare_hbr_models(dct)

Process: 94853 - 2025-08-18 10:37:20 - Dataset "synthesized" created.
    - 1078 observations
    - 1078 unique subjects
    - 1 covariates
    - 2 response variables
    - 2 batch effects:
    	sex (2)
	site (23)
    
Process: 94853 - 2025-08-18 10:37:20 - Synthesizing data for 2 response variables.
Process: 94853 - 2025-08-18 10:37:20 - Synthesizing data for Right-Lateral-Ventricle.
Process: 94853 - 2025-08-18 10:37:20 - Synthesizing data for WM-hypointensities.
Process: 94853 - 2025-08-18 10:37:21 - Making predictions on 2 response variables.
Process: 94853 - 2025-08-18 10:37:21 - Computing z-scores for 2 response variables.
Process: 94853 - 2025-08-18 10:37:21 - Computing z-scores for Right-Lateral-Ventricle.
Process: 94853 - 2025-08-18 10:37:21 - Computing z-scores for WM-hypointensities.
Process: 94853 - 2025-08-18 10:37:21 - Computing centiles for 2 response variables.
Process: 94853 - 2025-08-18 10:37:21 - Computing centiles for Right-Lateral-Ventricle.
Process: 94853 - 2025-08

Output()

Output()

/opt/anaconda3/envs/ptk/lib/python3.12/site-packages/arviz/stats/stats.py:797: UserWarning: Estimated shape parameter of Pareto distribution is greater than 0.70 for one or more samples. You should consider using a more robust model, this is because importance sampling is less likely to work well if the marginal posterior and LOO posterior are very different. This is more likely to happen with a non-robust model and highly influential observations.
  warnings.warn(


Output()

/opt/anaconda3/envs/ptk/lib/python3.12/site-packages/arviz/stats/stats.py:797: UserWarning: Estimated shape parameter of Pareto distribution is greater than 0.70 for one or more samples. You should consider using a more robust model, this is because importance sampling is less likely to work well if the marginal posterior and LOO posterior are very different. This is more likely to happen with a non-robust model and highly influential observations.
  warnings.warn(


Output()

/opt/anaconda3/envs/ptk/lib/python3.12/site-packages/arviz/stats/stats.py:1057: RuntimeWarning: overflow encountered in exp
  weights = 1 / np.exp(len_scale - len_scale[:, None]).sum(axis=1)
/opt/anaconda3/envs/ptk/lib/python3.12/site-packages/arviz/stats/stats.py:797: UserWarning: Estimated shape parameter of Pareto distribution is greater than 0.70 for one or more samples. You should consider using a more robust model, this is because importance sampling is less likely to work well if the marginal posterior and LOO posterior are very different. This is more likely to happen with a non-robust model and highly influential observations.
  warnings.warn(
/opt/anaconda3/envs/ptk/lib/python3.12/site-packages/arviz/stats/stats.py:1057: RuntimeWarning: overflow encountered in exp
  weights = 1 / np.exp(len_scale - len_scale[:, None]).sum(axis=1)
/opt/anaconda3/envs/ptk/lib/python3.12/site-packages/arviz/stats/stats.py:797: UserWarning: Estimated shape parameter of Pareto distribution is grea

In [8]:
for k, v in comparison.items():
    print(k)
    display(v)

Right-Lateral-Ventricle


,rank,elpd_loo,p_loo,elpd_diff,weight,se,dse,warning,scale
model2,0,-1845.230884,208.777051,0.000000,0.550206,57.537324,0.000000,True,log
model1,1,-1854.487416,150.538526,9.256532,0.449794,43.342593,72.566506,True,log


WM-hypointensities


,rank,elpd_loo,p_loo,elpd_diff,weight,se,dse,warning,scale
model2,0,-1742.644987,165.290704,0.00000,0.625097,48.956539,0.000000,True,log
model1,1,-1972.681637,164.645110,230.03665,0.374903,54.080989,73.073796,True,log
